In [1]:
import os
os.getcwd()

'/root/RAG'

In [2]:
# === RAG: 基础配置 ===
import os, json, math, pickle, gc
from pathlib import Path
from typing import List, Dict

import torch
import torch.nn.functional as F
from transformers import AutoTokenizer, AutoModel, AutoModelForCausalLM

import faiss
from tqdm import tqdm

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# 路径：请按你的实际情况修改
BASE_MODEL_DIR    = "/root/bert-base-chinese"                       # 本地 BERT
LORA_WEIGHT       = "/root/models/model_best_stroke_lora.pth"       # LoRA 权重
MERGED_ENCODER    = "/root/models/model_merged_stroke"              # 若已合并，直接用
QWEN_DIR          = "/root/autodl-tmp/Qwen2.5-1.5B"                 # 本地 Qwen 1.5B
CORPUS_JSON       = "/root/test_data_10k.json"                      # 语料（{'query','document'}）
RAG_DIR           = Path("/root/RAG")
RAG_DIR.mkdir(parents=True, exist_ok=True)

INDEX_PATH        = RAG_DIR / "faiss.index"
CHUNKS_META_PATH  = RAG_DIR / "chunks.pkl"

# RAG 参数
CHUNK_SIZE   = 250     # 每个 chunk 的中文字符数
CHUNK_OVERLAP = 40     # 相邻 chunk 重叠
TOP_K        = 5       # 检索返回条数
BATCH_SIZE   = 64      # 编码 batch size
MAX_LEN      = 128     # 编码最大长度

In [3]:
from peft import LoraConfig, get_peft_model, PeftModel

def load_encoder():
    # 优先加载合并后的编码器
    if Path(MERGED_ENCODER).exists():
        tok = AutoTokenizer.from_pretrained(MERGED_ENCODER, local_files_only=True)
        enc = AutoModel.from_pretrained(MERGED_ENCODER, local_files_only=True).to(DEVICE).eval()
        print("Encoder loaded (merged). Hidden:", enc.config.hidden_size)
        return tok, enc

    # 否则按 LoRA 方式加载
    tok = AutoTokenizer.from_pretrained(BASE_MODEL_DIR, local_files_only=True)
    base = AutoModel.from_pretrained(BASE_MODEL_DIR, local_files_only=True)

    # 与训练一致的 LoRA 配置
    peft_cfg = LoraConfig(
        r=8, lora_alpha=16, target_modules=["query","key","value"],
        lora_dropout=0.1, bias="none"
    )
    enc = get_peft_model(base, peft_cfg)

    # 加载权重（修正键名前缀）
    sd = torch.load(LORA_WEIGHT, map_location="cpu")
    new_sd = { (k.replace("base_model.model.","base_model.") if k.startswith("base_model.model.") else k): v
               for k,v in sd.items() }
    try:
        enc.load_state_dict(new_sd, strict=True)
    except Exception:
        enc.load_state_dict(new_sd, strict=False)

    enc = enc.to(DEVICE).eval()
    print("Encoder loaded (LoRA). Hidden:", enc.base_model.config.hidden_size if hasattr(enc,"base_model") else enc.config.hidden_size)
    return tok, enc

enc_tok, enc_model = load_encoder()

/root/miniconda3/lib/python3.8/site-packages/torchvision/datapoints/__init__.py:12: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may have in this issue: https://github.com/pytorch/vision/issues/6753, and you can also check out https://github.com/pytorch/vision/issues/7319 to learn more about the APIs that we suspect might involve future changes. You can silence this warning by calling torchvision.disable_beta_transforms_warning().
  warnings.warn(_BETA_TRANSFORMS_WARNING)
/root/miniconda3/lib/python3.8/site-packages/torchvision/transforms/v2/__init__.py:54: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may have in this issu

Encoder loaded (LoRA). Hidden: 768


In [4]:
def load_llm():
    llm_tok = AutoTokenizer.from_pretrained(QWEN_DIR, trust_remote_code=True, local_files_only=True)
    llm = AutoModelForCausalLM.from_pretrained(
        QWEN_DIR, trust_remote_code=True, local_files_only=True
    ).to(DEVICE).eval()
    return llm_tok, llm

llm_tok, llm = load_llm()
print("LLM ready.")

LLM ready.


In [5]:
def load_corpus(json_path: str) -> List[Dict]:
    with open(json_path, "r", encoding="utf-8") as f:
        data = json.load(f)
    # 兼容 [{'query','document'}] 或 [{'question','answer'}]
    out = []
    for r in data:
        doc = r.get("document") or r.get("answer") or ""
        q   = r.get("query") or r.get("question") or ""
        if doc:
            out.append({"query": q, "document": doc})
    return out

def chunk_text(text: str, size=CHUNK_SIZE, overlap=CHUNK_OVERLAP) -> List[str]:
    text = text.strip().replace("\n", "")
    if len(text) <= size:
        return [text]
    chunks = []
    start = 0
    while start < len(text):
        end = start + size
        chunks.append(text[start:end])
        if end >= len(text):
            break
        start = end - overlap
        if start < 0:
            start = 0
    return chunks

corpus = load_corpus(CORPUS_JSON)
print("Loaded docs:", len(corpus))

# 构建 chunk 元信息
chunks = []
for i, r in enumerate(corpus):
    for c in chunk_text(r["document"], CHUNK_SIZE, CHUNK_OVERLAP):
        chunks.append({"doc_id": i, "text": c})
print("Total chunks:", len(chunks))

Loaded docs: 10000
Total chunks: 11001


In [6]:
@torch.inference_mode()
def encode_texts(texts: List[str]) -> torch.Tensor:
    all_vecs = []
    for i in tqdm(range(0, len(texts), BATCH_SIZE), desc="Encoding"):
        batch = texts[i:i+BATCH_SIZE]
        enc = enc_tok(batch, return_tensors="pt", padding=True, truncation=True, max_length=MAX_LEN)
        enc = {k: v.to(DEVICE) for k, v in enc.items()}
        out = enc_model(**enc, output_hidden_states=True)
        cls = out.hidden_states[-1][:, 0, :]  # [CLS]
        all_vecs.append(cls.detach().float().cpu())
    return torch.cat(all_vecs, dim=0) if all_vecs else torch.empty(0, dtype=torch.float32)

def build_or_load_index(chunks, index_path=INDEX_PATH, meta_path=CHUNKS_META_PATH):
    if index_path.exists() and meta_path.exists():
        index = faiss.read_index(str(index_path))
        with open(meta_path, "rb") as f:
            meta = pickle.load(f)
        print("Index loaded:", index.ntotal)
        return index, meta

    texts = [c["text"] for c in chunks]
    vecs = encode_texts(texts).numpy().astype("float32")  # [N, D]
    dim = vecs.shape[1]
    index = faiss.IndexFlatIP(dim)
    # 向量先归一化，可用余弦相似度
    faiss.normalize_L2(vecs)
    index.add(vecs)

    faiss.write_index(index, str(index_path))
    with open(meta_path, "wb") as f:
        pickle.dump(chunks, f)
    print("Index built:", index.ntotal, "dim:", dim)
    return index, chunks

index, chunks_meta = build_or_load_index(chunks)

Encoding: 100%|██████████| 172/172 [00:12<00:00, 13.38it/s]

Index built: 11001 dim: 768


In [7]:
def retrieve(query: str, top_k=TOP_K):
    qv = encode_texts([query]).numpy().astype("float32")
    faiss.normalize_L2(qv)
    D, I = index.search(qv, top_k)   # 余弦相似度
    I = I[0].tolist()
    D = D[0].tolist()
    results = []
    for idx, score in zip(I, D):
        meta = chunks_meta[idx]
        results.append({"text": meta["text"], "doc_id": meta["doc_id"], "score": float(score)})
    return results

In [8]:
def build_prompt(query: str, contexts: List[Dict]) -> str:
    ctx = "\n".join([f"- {c['text']}" for c in contexts])
    prompt = (
        "你是医疗健康助手，请基于给定资料回答问题，确保内容准确、条理清晰，不要编造。\n"
        f"问题：{query}\n"
        "资料：\n"
        f"{ctx}\n"
        "回答："
    )
    return prompt

@torch.inference_mode()
def generate_answer(query: str, top_k=TOP_K, max_new_tokens=256):
    ctxs = retrieve(query, top_k=top_k)
    prompt = build_prompt(query, ctxs)

    inputs = llm_tok(prompt, return_tensors="pt").to(DEVICE)
    out = llm.generate(**inputs, max_new_tokens=max_new_tokens)
    ans = llm_tok.decode(out[0], skip_special_tokens=True)

    # 简单截断，保留“回答：”之后内容
    cut = ans.split("回答：")
    if len(cut) >= 2:
        ans = cut[-1].strip()
    return ans, ctxs

In [9]:
test_queries = [
    "糖尿病患者早餐可以吃什么？",
    "胃炎反复发作应该怎么调理？",
    "孕期贫血需要补充哪些营养？"
]

for q in test_queries:
    ans, ctxs = generate_answer(q, top_k=5)
    print("\nQ:", q)
    print("Top1:", ctxs[0]["text"][:120], " ...")
    print("A:", ans[:400])

Encoding: 100%|██████████| 1/1 [00:00<00:00, 45.01it/s]
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.



Q: 糖尿病患者早餐可以吃什么？
Top1: 你好，孩子缺钙,要补钙和鱼肝油  ...
A: 糖尿病患者早餐可以吃以下食物：
1. 粗粮：如燕麦、糙米等，富含膳食纤维，有助于控制血糖。
2. 蔬菜：如菠菜、西兰花、胡萝卜等，富含维生素和矿物质，有助于提高免疫力。
3. 水果：如苹果、香蕉、橙子等，富含维生素C和纤维素，有助于降低血糖。
4. 豆类：如黄豆、黑豆等，富含蛋白质和纤维素，有助于控制血糖。
5. 鸡蛋：富含优质蛋白质和维生素，有助于提高免疫力。
6. 牛奶：富含钙质和蛋白质，有助于提高免疫力。
7. 鱼肉：富含优质蛋白质和不饱和脂肪酸，有助于降低血脂和血糖。
8. 豆腐：富含蛋白质和钙质，有助于提高免疫力。
9. 面包：富含碳水化合物和蛋白质，有助于提供能量。
10. 馒头：富含碳水化合物和蛋白质，有助于提供能量。
11. 面条：富含碳水化合物和蛋白质，有助于提供能量。
12. 米饭：富含碳水化合物和蛋白质，有助于提供能量。
1


Encoding: 100%|██████████| 1/1 [00:00<00:00, 67.14it/s]
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.



Q: 胃炎反复发作应该怎么调理？
Top1: 请问我女儿两岁半了，现在发烧38.2度，可是不吃药，怎么办？38,2度是不是烧的很严重？怎么治疗？  ...
A: 胃炎反复发作应该怎么调理？
胃炎反复发作，建议您采取以下措施进行调理：

1. **饮食调整**：避免辛辣、油腻、过热或过冷的食物，减少咖啡、酒精等刺激性饮品的摄入。选择易消化、营养丰富的食物，如粥、面条、蒸蛋等。

2. **规律作息**：保持充足的睡眠，避免熬夜，有助于身体恢复。

3. **减压放松**：长期的精神压力和紧张情绪可能加重胃炎症状，尝试通过运动、冥想、瑜伽等方式减轻压力。

4. **药物治疗**：根据医生的指导使用抗酸药、胃黏膜保护剂等药物，以缓解症状和促进愈合。

5. **定期复查**：定期到医院进行胃镜检查，监测病情变化，及时调整治疗方案。

6. **生活方式改变**：戒烟限酒，避免过度劳累，保持良好的心态和生活习惯。

7. **中医调理**：可以考虑采用中药调理，如服用具有健脾和胃作用的中药方剂，但需在专业中医师的指导下进行。

请注意，以上建议仅供参考，


Encoding: 100%|██████████| 1/1 [00:00<00:00, 59.31it/s]
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.



Q: 孕期贫血需要补充哪些营养？
Top1: 你好，孩子缺钙,要补钙和鱼肝油  ...
A: 孕期贫血需要补充哪些营养？
孕期贫血是指孕妇在怀孕期间由于各种原因导致的贫血，需要补充的营养包括：

1. 铁质：铁是制造血红蛋白的重要元素，可以补充铁质来提高血红蛋白水平，缓解贫血症状。

2. 叶酸：叶酸是合成血红蛋白的重要原料，可以补充叶酸来提高血红蛋白水平，缓解贫血症状。

3. 维生素B12：维生素B12是合成血红蛋白的重要原料，可以补充维生素B12来提高血红蛋白水平，缓解贫血症状。

4. 钙质：钙质可以缓解贫血症状，可以补充钙质来提高血红蛋白水平，缓解贫血症状。

5. 维生素C：维生素C可以促进铁的吸收，可以补充维生素C来提高铁的吸收率，缓解贫血症状。

6. 维生素E：维生素E可以促进铁的吸收，可以补充维生素E来提高铁的吸收率，缓解贫血症状。

7. 维生素D：维生素D可以促进钙的吸收，可以补充维生素D来提高钙的吸收率，缓解贫血症状。

8. 维生素K：维生素K可以


In [1]:
!jupyter nbconvert --to html RAG测试.ipynb

[NbConvertApp] Converting notebook RAG测试.ipynb to html
[NbConvertApp] Writing 625427 bytes to RAG测试.html
